# Process an interferogram with ASF HyP3

https://hyp3-docs.asf.alaska.edu/using/sdk/ 

## Search for scenes

scenes over grand mesa, colorado using https://asf.alaska.edu/api/

In [17]:
import requests
import shapely.geometry

roi = shapely.geometry.box(-106.566,35.688,-106.304,36.121)
polygonWKT = roi.wkt

baseurl = "https://api.daac.asf.alaska.edu/services/search/param"

data = dict(
    intersectsWith=polygonWKT,
    platform='Sentinel-1',
    processingLevel="SLC",
    beamMode='IW',
    output='json',
    start='2021-02-19T11:59:59Z',
    end='2021-03-08T11:59:59Z',
    #relativeOrbit=None,
    #flightDirection=None,
)

r = requests.get(baseurl, params=data, timeout=100)
print(r.url)

https://api.daac.asf.alaska.edu/services/search/param?intersectsWith=POLYGON+%28%28-106.304+35.688%2C+-106.304+36.121%2C+-106.566+36.121%2C+-106.566+35.688%2C+-106.304+35.688%29%29&platform=Sentinel-1&processingLevel=SLC&beamMode=IW&output=json&start=2021-02-19T11%3A59%3A59Z&end=2021-03-08T11%3A59%3A59Z


In [14]:
# load results into pandas dataframe
import pandas as pd
df = pd.DataFrame(r.json()[0])
display(df)

,absoluteOrbit,beamMode,beamModeType,beamSwath,browse,catSceneId,centerLat,centerLon,collectionName,configurationName,...,sensor,sizeMB,slaveGranule,startTime,status,stopTime,stringFootprint,thumbnailUrl,track,varianceTroposphere
0,25902,IW,IW,None,None,None,36.4669,-105.6039,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,4096.6252155303955,None,2021-03-07T01:00:52.000000,None,2021-03-07T01:01:19.000000,"POLYGON((-104.349129 37.469887,-104.021988 35....",None,151,None
1,36878,IW,IW,None,None,None,35.1212,-107.2677,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,3868.169867515564,None,2021-03-06T13:11:03.000000,None,2021-03-06T13:11:30.000000,"POLYGON((-108.808792 34.502563,-108.483223 36....",None,56,None
2,36878,IW,IW,None,None,None,36.6425,-106.9264,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,4161.684388160706,None,2021-03-06T13:10:37.000000,None,2021-03-06T13:11:05.000000,"POLYGON((-108.504555 35.994625,-108.164185 37....",None,56,None
3,36805,IW,IW,None,None,None,34.7292,-105.2902,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,3746.820252418518,None,2021-03-01T13:02:55.000000,None,2021-03-01T13:03:22.000000,"POLYGON((-106.824287 34.111134,-106.499451 35....",None,158,None
4,36805,IW,IW,None,None,None,36.2521,-104.9638,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,4063.6540145874023,None,2021-03-01T13:02:30.000000,None,2021-03-01T13:02:58.000000,"POLYGON((-106.530579 35.604019,-106.191231 37....",None,158,None
5,25800,IW,IW,None,None,None,35.5756,-107.4815,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,3953.4769229888916,None,2021-02-28T01:08:51.000000,None,2021-02-28T01:09:18.000000,"POLYGON((-106.257790 36.577381,-105.931976 34....",None,49,None
6,25727,IW,IW,None,None,None,36.4669,-105.6039,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,4123.9064083099365,None,2021-02-23T01:00:53.000000,None,2021-02-23T01:01:20.000000,"POLYGON((-104.349236 37.470016,-104.022041 35....",None,151,None
7,36703,IW,IW,None,None,None,35.1214,-107.268,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,3863.8349323272705,None,2021-02-22T13:11:03.000000,None,2021-02-22T13:11:30.000000,"POLYGON((-108.809189 34.502728,-108.483635 36....",None,56,None
8,36703,IW,IW,None,None,None,36.6425,-106.9268,None,"Interferometric Wide. 250 km swath, 5 m x 20 m...",...,C-SAR,4168.028311729431,None,2021-02-22T13:10:37.000000,None,2021-02-22T13:11:05.000000,"POLYGON((-108.504982 35.994659,-108.164658 37....",None,56,None


In [15]:
# Easier to explore the inventory in plots
import hvplot.pandas
from bokeh.models.formatters import DatetimeTickFormatter

formatter = DatetimeTickFormatter(years='%m-%d')
timeseries = df.hvplot.scatter(x='startTime', y='relativeOrbit', c='relativeOrbit',
                               xformatter=formatter,
                               title='Acquisition times (UTC)')

In [18]:
import geopandas as gpd
import geoviews as gv
import panel as pn

gf_aoi = gpd.GeoDataFrame(geometry=[roi])
polygons = df.stringFootprint.apply(shapely.wkt.loads)
gf_footprints = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=polygons)

tiles = gv.tile_sources.StamenTerrainRetina.options(width=600, height=400)
aoi = gf_aoi.hvplot(geo=True, fill_color=None, line_color='m', hover=False)
footprints = gf_footprints.hvplot.polygons(geo=True, legend=False, alpha=0.2, c='relativeOrbit', title='Sentinel-1 Tracks') 

mapview = tiles * footprints * aoi

pn.Column(mapview,timeseries)

Column
    [0] HoloViews(Overlay)
    [1] HoloViews(Scatter)

In [16]:
df.relativeOrbit.unique()

array(['151', '56', '158', '49'], dtype=object)

In [24]:
orbit = '151'
reference = '2021-02-23'
secondary = '2021-03-07'

dfS = df[df.relativeOrbit == orbit]
granule1 = dfS.loc[dfS.sceneDate.str.startswith(reference), 'granuleName'].values[0]
granule2 = dfS.loc[dfS.sceneDate.str.startswith(secondary), 'granuleName'].values[0]
print(f'granule1: {granule1}')
print(f'granule2: {granule2}')

granule1: S1B_IW_SLC__1SDV_20210223T010053_20210223T010120_025727_031125_0EF6
granule2: S1B_IW_SLC__1SDV_20210307T010052_20210307T010119_025902_0316DD_12D1


In [25]:
for ref in [reference, secondary]:
    print(dfS.loc[dfS.sceneDate.str.startswith(ref), 'downloadUrl'].values[0])

https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210223T010053_20210223T010120_025727_031125_0EF6.zip
https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210307T010052_20210307T010119_025902_0316DD_12D1.zip


## Process an InSAR pair (interferogram)

examples:
- https://nbviewer.jupyter.org/github/ASFHyP3/hyp3-sdk/blob/main/docs/sdk_example.ipynb
- https://hyp3-docs.asf.alaska.edu/using/sdk/

In [26]:
import hyp3_sdk

In [27]:
# ~/.netrc file used for credentials
hyp3 = hyp3_sdk.HyP3()

In [28]:
# Processing quota
hyp3.check_quota() #199 (200 scenes per month?)

998

In [30]:
job2 = hyp3.submit_insar_job(granule1,
                            granule2,
                            name='jemez_20200222_20200305')

KeyError: 'detail'

In [ ]:
# All jobs you've submitted
# NOTE: processing w/ defaults uses INSAR_GAMMA 
# NOTE: re-run this cell to update results of batch job
batch = hyp3.find_jobs()
job = batch.jobs[0] # most recent job
job

In [ ]:
# If you have lists of dictionaries, visualizing with a pandas dataframe is convenient
df = pd.DataFrame([job.to_dict() for job in batch])
df.head()

In [ ]:
# Actually no, expiration time is not available for download...
#pd.to_datetime(df.expiration_time[0]) - pd.to_datetime(df.request_time[0])

In [ ]:
# requires ipywidgets
hyp3.watch(job)

In [ ]:
# ImportError: IProgress not found. Please update jupyter and ipywidgets.
# but I think this still succeeeds
job.download_files()

In [ ]:
!ls -ltrh

## Process multiple pairs in batch mode

In [ ]:
# with progress bar
#from tqdm.auto import tqdm 

#insar_jobs = sdk.Batch()
#for reference in tqdm(granules):
#    neighbors_metadata = asf_search.get_nearest_neighbors(reference, max_neighbors=2)
#    for secondary_metadata in neighbors_metadata:
#        insar_jobs += hyp3.submit_insar_job(reference, secondary_metadata['granuleName'], name='insar-example')
#print(insar_jobs)

In [ ]:
# Can also submit jobs via web interface # Can also visit https://hyp3.asf.alaska.edu/pending_products 
# Which then shows logs that can be sorted into 'submitted, failed, etc...'